<a href="https://colab.research.google.com/github/Asm3515/DataScience---MLP/blob/main/MLP_Sentiment_Predection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import zipfile
zip_file = '/content/archive (11).zip'
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall('/content/Extracted_Data')


In [ ]:
import csv
import pandas as pd

df = pd.read_csv(r"/content/Extracted_Data/IMDB Dataset.csv",encoding='utf-8')
df.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd

def save_train_test_files(data_file_name, test_size):
    data = pd.read_csv(data_file_name)
    train_data, test_data = train_test_split(data, test_size=test_size/100)
    train_file_name = f"train_{data_file_name.split('/')[-1].split('.')[0]}_{test_size}.csv"
    test_file_name = f"test_{data_file_name.split('/')[-1].split('.')[0]}_{test_size}.csv"
    train_data.to_csv(train_file_name, index=False)
    test_data.to_csv(test_file_name, index=False)

save_train_test_files('/content/Extracted_Data/IMDB Dataset.csv', 20)


In [ ]:
def load_imdb_dataset(path_to_dataset):
    imdb_data = pd.read_csv(path_to_dataset)  # Read the dataset
    features = imdb_data['review']  # Extracting the features
    targets = imdb_data['sentiment']  # Extracting the target labels
    return features, targets


In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder
import gensim.downloader

model = gensim.downloader.load('word2vec-google-news-300')

def train_MLP_model_average(path_to_train_file):
    X, y = load_imdb_dataset(path_to_train_file)
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y)

    def average_word_embeddings(text):
        words = text.split()
        embeddings = [model[word] for word in words if word in model]
        return np.mean(embeddings, axis=0) if embeddings else np.zeros(model.vector_size)

    X_avg = np.array([average_word_embeddings(text) for text in X])

    mlp_model = MLPClassifier(hidden_layer_sizes=(100, 100, 100), max_iter=1000, random_state=42)
    mlp_model.fit(X_avg, y)

    return mlp_model


[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder
import gensim.downloader
import numpy as np

model = gensim.downloader.load('word2vec-google-news-300')

def train_MLP_model_student(path_to_train_file):
    X, y = load_imdb_dataset(path_to_train_file)
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y)

    def sum_word_embeddings(text):
        words = text.split()
        embeddings = [model[word] for word in words if word in model]
        return np.sum(embeddings, axis=0) if embeddings else np.zeros(model.vector_size)

    X_sum = np.array([sum_word_embeddings(text) for text in X])

    mlp_model = MLPClassifier(hidden_layer_sizes=(100, 100, 100), max_iter=1000, random_state=42)
    mlp_model.fit(X_sum, y)

    return mlp_model


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.metrics import accuracy_score

model = gensim.downloader.load('word2vec-google-news-300')

def test_MLP_model(path_to_test_file, MLP_model, input='average'):
    test_df = pd.read_csv(path_to_test_file)
    test_X = test_df['review']
    y_test = test_df['sentiment']

    label_encoder = LabelEncoder()
    y_test = label_encoder.fit_transform(y_test)

    def average_word_embeddings(text):
        words = text.split()
        embeddings = [model[word] for word in words if word in model]
        return np.mean(embeddings, axis=0) if embeddings else np.zeros(model.vector_size)

    def sum_word_embeddings(text):
        words = text.split()
        embeddings = [model[word] for word in words if word in model]
        return np.sum(embeddings, axis=0) if embeddings else np.zeros(model.vector_size)

    if input == 'average':
        test_X_avg = np.array([average_word_embeddings(text) for text in test_X])
        test_predictions = MLP_model.predict(test_X_avg)
        test_probabilities = MLP_model.predict_proba(test_X_avg)
    else:
        test_X_sum = np.array([sum_word_embeddings(text) for text in test_X])
        test_predictions = MLP_model.predict(test_X_sum)
        test_probabilities = MLP_model.predict_proba(test_X_sum)

    test_df['probability_positive'] = test_probabilities[:, 1]
    test_df['class_prediction'] = ['positive' if pred == 1 else 'negative' for pred in test_predictions]

    test_df.to_csv(f"{input}_output_test_file.csv", index=False)

    acc = accuracy_score(y_test, test_predictions)
    print(f"Accuracy for {input} embedding: {acc}")


In [ ]:
average_model=train_MLP_model_average("/content/train_IMDB Dataset_20.csv")
student_model=train_MLP_model_student("/content/train_IMDB Dataset_20.csv")

In [ ]:
test_path = r'/content/test_IMDB Dataset_20.csv'
test_MLP_model(test_path,average_model)
test_MLP_model(test_path,student_model)



Accuracy for average embedding: 0.8142
Accuracy for average embedding: 0.4963
